# Data Analytics and Data Mining
## The effects of weather on agricoltural commodity price predictions



In [1]:
#Pip Packages
!pip install pmdarima
!pip install statsmodels==0.11.0

#Imports and settings
import numpy as np, pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import matplotlib.dates
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf #Metrics
import pmdarima as pm #Auto arima

#Settings
%config InlineBackend.figure_format='retina'

### Retrieve data from a private remote server

In [ ]:
remote_host = 'https://www.ingmarzo.it/data/'
corn_data = pd.read_csv(remote_host + 'corn-data.csv', header=8, parse_dates=['date'])
soybean_data = pd.read_csv(remote_host + 'soybean-data.csv', header=8, parse_dates=['date'])
weather_data = pd.read_csv(remote_host + 'weather-data.csv', header=0, parse_dates=['DATE'])

## Data Cleaning

In [ ]:
corn_data.rename(columns={' value': 'value'}, inplace=True) #bug on value fixed
weather_data.rename(columns={'DATE': 'date'}, inplace=True) #rename date column
corn_data['date'] = pd.to_datetime(corn_data['date'])  

start_date, end_date = '2011-01-01', '2015-01-01'
corn_date_filter = (corn_data['date'] > start_date) & (corn_data['date'] <= end_date)

corn_data_clean = corn_data.loc[corn_date_filter]

## Univariate Time Series Forecasting
Use only the past values of the time series to predict its future values.

---


**ARIMA** i.e. Auto Regressive Integrated Moving Average, is a class of models that analyses a time series based on its own previous values and can be used to anticipate future values. 

### Auto regressive (AR) Analysis
To avoid the effect of inter-dependency it is possible to apply differentiation and make the series Stationary.

In [ ]:
# Is the series stationary?
# ADF Test
result = adfuller(corn_data_clean['value'].dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

### Test results
Since P-value is greater than the significance level, the series is autocorrelated (as expected). It is now possible to differentiate the series and repeat the ADF Test

In [ ]:
plt.rcParams.update({'figure.figsize':(16,8)})

# Original Series
fig, axes = plt.subplots(2, 2, sharex=True)
axes[0, 0].plot(corn_data_clean['value'].dropna().values)
axes[0, 0].set_title('Original Series')
plot_acf(corn_data_clean['value'].dropna().values, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(corn_data_clean['value'].diff().values)
axes[1, 0].set_title('1st Order Differencing')
plot_acf(corn_data_clean['value'].diff().dropna().values, ax=axes[1, 1])

plt.show()


In [ ]:
# Test autocorrelation for 1st order differencing
result = adfuller(corn_data_clean['value'].diff().dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

### Corn price is no longer autocorrelated
The resulting series `corn_data_clean['value'].diff()` has been proven stationary

In [ ]:
#corn_data_clean['value'].diff().values

### Partial autocorrelation
Many values can be still partially autocorrelated, the PACF plot gives a visual hint about how resudual partial autocorrelation is present in the data. Since many values resides within the lightblue region (Significance limit region) the effect of autocorrelation is well contained. Values outside of the lightblue region are the most affected by autocoreelation

In [ ]:
plt.rcParams.update({'figure.figsize':(16,4)})
fig, axes = plt.subplots(1, 1, sharex=True)
plot_pacf(corn_data_clean['value'].diff().dropna(), ax=axes)
plt.show()

## Import and set up ARIMA Model

In [ ]:
#ARIMA Model
model = ARIMA(corn_data_clean['value'].dropna().values, order=(1,1,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())

### Visualize residual errors

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

### Is the error consistent?
Test if residual errors follow normal distribution

In [ ]:
import scipy.stats as stats

#perform Chi-Square Goodness of Fit Test
#H0: residual error follows normal distribution
#H1: residual error does not follow normal distribution
hyp_test = stats.normaltest(residuals.sample(100, random_state=19).values)
res = 'Price difference does ' + ('' if hyp_test[1] > 0.05 else 'not ') +\
        f'follow the normal distribution, p-value: {hyp_test[1]}'
print(f'Residuals variance: {np.var(residuals).values} \
        mean: {np.mean(residuals).values}')
print(res)

### Plot actual vs predicted values

In [ ]:
plt.rcParams.update({'figure.figsize':(16,6)})
model_fit.plot_predict(dynamic=False)
plt.show()

## Out-of-time ARIMA validation
In order to tune the model to predict unseen prices it is possible to split the actual dataset in train (75%) and test (25%) subsets.  
It is then possible to fit ARIMA model with the training test and compare predicted results with test set.

In [ ]:
perc_split = .85
corn_data_split = round(len(corn_data_clean) * perc_split)
corn_data_train = corn_data_clean.iloc[0:corn_data_split]
corn_data_test = corn_data_clean.iloc[corn_data_split:]

### Plot predictions

In [ ]:
model = ARIMA(corn_data_train['value'].dropna().values, order=(1, 2, 1))  
fitted = model.fit(disp=-1)  

# Forecast
fc, se, conf = fitted.forecast(len(corn_data_test), alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=corn_data_test['value'].index)
lower_series = pd.Series(conf[:, 0], index=corn_data_test['value'].index)
upper_series = pd.Series(conf[:, 1], index=corn_data_test['value'].index)

# Plot
plt.rcParams.update({'figure.figsize':(16, 6)})
plt.plot(corn_data_train['value'], label='training')
plt.plot(corn_data_test['value'], label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

### Empirical evaluation
Results are inconsistent with actual data since the majority of results are below the actual price. This lead to find best arima parameters to train a more precise model.

In [ ]:
model = pm.auto_arima(corn_data_train['value'].dropna(), start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

#print(model.summary())
model.aic()

## Plot ARIMA diagnostic

In [ ]:
plt.rcParams.update({'figure.figsize':(16,8)})
model.plot_diagnostics()
plt.show()

## Accuracy Metrics for Time Series Predictions

In [ ]:
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(forecast - actual)[1]            # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(fc, corn_data_test['value'].values)

## Use best ARIMA model to compute predictions

In [ ]:
model = ARIMA(corn_data_train['value'].dropna().values, order=(1, 2, 1))  
fitted = model.fit(disp=-1)  

# Forecast
fc, se, conf = fitted.forecast(len(corn_data_test), alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=corn_data_test['value'].index)
lower_series = pd.Series(conf[:, 0], index=corn_data_test['value'].index)
upper_series = pd.Series(conf[:, 1], index=corn_data_test['value'].index)

# Plot
plt.plot(corn_data_train['value'], label='training')
plt.plot(corn_data_test['value'], label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()



## Implement Seasonality Analysis
Compute differentiation with respect to the previous season values.

In [ ]:
# Seasonal period: year
diff_factor = 365

# Plot
fig, axes = plt.subplots(2, 1, sharex=True)

# Usual Differencing
axes[0].plot(corn_data_clean['value'].values, label='Original Series')
axes[0].plot(corn_data_clean['value'].diff(1).values, label='Usual Differencing')
axes[0].set_title('Usual Differencing')
axes[0].legend(loc='upper left', fontsize=10)


# Seasinal Dei
axes[1].plot(corn_data_clean['value'].values, label='Original Series')
axes[1].plot(corn_data_clean['value'].diff(diff_factor).values, label='Seasonal Differencing', color='green')
axes[1].set_title('Seasonal Differencing')
plt.legend(loc='upper left', fontsize=10)
plt.suptitle('Corn price time series', fontsize=16)
plt.show()

In [ ]:
smodel = pm.auto_arima(corn_data_clean['value'].values, start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=diff_factor,
                         start_P=0, seasonal=True,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

smodel.summary()